In [3]:
pip install skyfield plotly requests pandas numpy

   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ---------------------------- ----------- 7.1/9.8 MB 33.6 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 29.7 MB/s  0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------  10.7/11.0 MB 54.5 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 40.8 MB/s  0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------  12.6/12.8 MB 67.2 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 42.7 MB/s  0:00:00

   ---------------------------------------- 0/9 [sgp4]
   ---------------------------------------- 0/9 [sgp4]
   ---------------------------------------- 0/9 [sgp4]
   ---- ----------------------------------- 1/9 [pytz]
   ---- ----------------------------------- 1/9 [pytz]
   ---- ----------------------------------- 1/9 [pytz]

In [4]:
import skyfield
import plotly
import requests
print("All packages imported successfully!")

All packages imported successfully!


In [10]:
import requests
import pandas as pd
from skyfield.api import load, EarthSatellite, wgs84
from datetime import datetime, timedelta
import numpy as np

# Get TLE data from CelesTrak (immediate access)
def fetch_satellite_data():
    url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"
    response = requests.get(url)
    
    satellites = []
    lines = response.text.strip().split('\n')
    
    ts = load.timescale()
    for i in range(0, len(lines)-2, 3):
        try:
            name = lines[i].strip()
            line1 = lines[i+1].strip()
            line2 = lines[i+2].strip()
            
            sat = EarthSatellite(line1, line2, name, ts)
            
            # Extract key orbital parameters
            mean_motion = float(line2[52:63])  # revolutions per day
            
            # Simplified altitude calculation from mean motion
            # Using the orbital period formula: T = 2π√(a³/μ)
            orbital_period_minutes = 1440 / mean_motion  # minutes
            altitude_km = ((orbital_period_minutes * 60)**2 * 398600.5 / (4 * np.pi**2))**(1/3) - 6371
            
            satellites.append({
                'name': name,
                'norad_id': line1[2:7],
                'inclination': float(line2[8:16]),
                'eccentricity': float('0.' + line2[26:33]),
                'mean_motion': mean_motion,
                'altitude_km': altitude_km,
                'tle_line1': line1,
                'tle_line2': line2
            })
        except:
            continue
    
    return pd.DataFrame(satellites)

In [11]:
def find_satellites_over_location(df_satellites, lat, lon, city_name, time_window_hours=24):
    """
    Find which satellites pass over a specific location
    """
    ts = load.timescale()
    location = wgs84.latlon(lat, lon)
    
    results = []
    current_time = ts.now()
    end_time = ts.utc(datetime.utcnow() + timedelta(hours=time_window_hours))
    
    for _, sat_data in df_satellites.iterrows():
        satellite = EarthSatellite(sat_data['tle_line1'], sat_data['tle_line2'], 
                                  sat_data['name'], ts)
        
        # Check if satellite passes over location
        t, events = satellite.find_events(location, current_time, end_time, 
                                         altitude_degrees=10.0)  # 10° above horizon
        
        if len(t) > 0:
            # Calculate minimum distance and maximum elevation
            for time in t:
                difference = satellite - location
                topocentric = difference.at(time)
                alt, az, distance = topocentric.altaz()
                
                results.append({
                    'city': city_name,
                    'lat': lat,
                    'lon': lon,
                    'satellite_name': sat_data['name'],
                    'norad_id': sat_data['norad_id'],
                    'pass_time': time.utc_datetime(),
                    'max_elevation_deg': alt.degrees,
                    'distance_km': distance.km,
                    'inclination': sat_data['inclination'],
                    'altitude_km': sat_data['altitude_km']
                })
    
    return pd.DataFrame(results)

In [12]:
def calculate_risk_factors(satellite_passes_df):
    """
    Calculate risk factors based on orbital characteristics
    """
    risk_data = []
    
    for city in satellite_passes_df['city'].unique():
        city_data = satellite_passes_df[satellite_passes_df['city'] == city]
        
        # Risk Factor 1: Congestion (number of satellites)
        congestion_score = len(city_data) / 100  # Normalize to 0-1
        
        # Risk Factor 2: Low altitude satellites (more debris risk)
        low_alt_sats = city_data[city_data['altitude_km'] < 600]
        debris_risk_score = len(low_alt_sats) / len(city_data) if len(city_data) > 0 else 0
        
        # Risk Factor 3: High inclination variation (crossing orbits)
        inclination_std = city_data['inclination'].std()
        crossing_risk = min(inclination_std / 90, 1)  # Normalize to 0-1
        
        # Composite risk score
        total_risk = (congestion_score * 0.4 + 
                     debris_risk_score * 0.4 + 
                     crossing_risk * 0.2)
        
        risk_data.append({
            'city': city,
            'lat': city_data['lat'].iloc[0],
            'lon': city_data['lon'].iloc[0],
            'satellite_count': len(city_data),
            'congestion_score': round(congestion_score, 3),
            'debris_risk_score': round(debris_risk_score, 3),
            'crossing_risk_score': round(crossing_risk, 3),
            'total_risk_score': round(total_risk, 3),
            'risk_level': 'High' if total_risk > 0.7 else 'Medium' if total_risk > 0.4 else 'Low'
        })
    
    return pd.DataFrame(risk_data)

In [14]:
from skyfield.api import load, EarthSatellite, wgs84, utc
from datetime import datetime, timedelta

def find_satellites_over_location(df_satellites, lat, lon, city_name, time_window_hours=24):
    """
    Find which satellites pass over a specific location
    """
    ts = load.timescale()
    location = wgs84.latlon(lat, lon)
    
    results = []
    # Fix: Use Skyfield's time methods directly
    current_time = ts.now()
    
    # Calculate end time properly
    now = datetime.now(utc)  # timezone-aware
    future = now + timedelta(hours=time_window_hours)
    end_time = ts.from_datetime(future)
    
    for _, sat_data in df_satellites.iterrows():
        satellite = EarthSatellite(sat_data['tle_line1'], sat_data['tle_line2'], 
                                  sat_data['name'], ts)
        
        # Check if satellite passes over location
        try:
            t, events = satellite.find_events(location, current_time, end_time, 
                                             altitude_degrees=10.0)  # 10° above horizon
            
            if len(t) > 0:
                # Calculate minimum distance and maximum elevation
                for time in t:
                    difference = satellite - location
                    topocentric = difference.at(time)
                    alt, az, distance = topocentric.altaz()
                    
                    results.append({
                        'city': city_name,
                        'lat': lat,
                        'lon': lon,
                        'satellite_name': sat_data['name'],
                        'norad_id': sat_data['norad_id'],
                        'pass_time': time.utc_datetime(),
                        'max_elevation_deg': alt.degrees,
                        'distance_km': distance.km,
                        'inclination': sat_data['inclination'],
                        'altitude_km': sat_data['altitude_km']
                    })
        except:
            # Skip satellites that cause computation errors
            continue
    
    return pd.DataFrame(results)

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create comparison dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Satellite Passes Over Cities', 'Risk Assessment',
                    'Orbital Altitude Distribution', 'Risk Factors Breakdown')
)

# Add traces for each subplot
# ... (visualization code)

fig.show()

In [17]:
import requests
import pandas as pd
import numpy as np
from skyfield.api import load, EarthSatellite, wgs84, utc
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Step 1: Fetch satellite data
print("Fetching satellite data...")
url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"
response = requests.get(url)

satellites = []
lines = response.text.strip().split('\n')
ts = load.timescale()

for i in range(0, len(lines)-2, 3):
    try:
        name = lines[i].strip()
        line1 = lines[i+1].strip()
        line2 = lines[i+2].strip()
        
        mean_motion = float(line2[52:63])
        orbital_period_minutes = 1440 / mean_motion
        altitude_km = ((orbital_period_minutes * 60)**2 * 398600.5 / (4 * np.pi**2))**(1/3) - 6371
        
        satellites.append({
            'name': name,
            'norad_id': line1[2:7],
            'inclination': float(line2[8:16]),
            'eccentricity': float('0.' + line2[26:33]),
            'mean_motion': mean_motion,
            'altitude_km': altitude_km
        })
    except:
        continue

df_satellites = pd.DataFrame(satellites)
print(f"Loaded {len(df_satellites)} satellites\n")

# Step 2: Define cities
cities = {
    'Philadelphia': {'lat': 39.9526, 'lon': -75.1652},
    'Rio de Janeiro': {'lat': -22.9068, 'lon': -43.1729}
}

# Step 3: Create risk assessment based on orbital characteristics
cities_risk = []
for city_name, coords in cities.items():
    lat = coords['lat']
    
    # Calculate risk factors
    leo_count = len(df_satellites[df_satellites['altitude_km'] < 2000])
    leo_risk = leo_count / 1000
    
    geo_visibility = 1 - (abs(lat) / 90)
    geo_count = len(df_satellites[df_satellites['altitude_km'] >= 35786])
    geo_risk = geo_count * geo_visibility / 100
    
    polar_sats = df_satellites[(df_satellites['inclination'] > 80) & 
                               (df_satellites['inclination'] < 100)]
    polar_risk = len(polar_sats) * (abs(lat) / 90) / 100
    
    total_risk = (leo_risk * 0.5 + geo_risk * 0.3 + polar_risk * 0.2)
    
    cities_risk.append({
        'City': city_name,
        'Latitude': lat,
        'Longitude': coords['lon'],
        'LEO Risk Factor': round(leo_risk, 3),
        'GEO Visibility Factor': round(geo_risk, 3),
        'Polar Coverage Factor': round(polar_risk, 3),
        'Total Risk Score': round(total_risk, 3),
        'Risk Level': 'High' if total_risk > 0.5 else 'Medium' if total_risk > 0.3 else 'Low'
    })

risk_df = pd.DataFrame(cities_risk)
print("City Risk Assessment:")
print(risk_df[['City', 'LEO Risk Factor', 'GEO Visibility Factor', 
               'Polar Coverage Factor', 'Total Risk Score', 'Risk Level']].to_string())
print("\n")

# Step 4: Create comprehensive dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Orbital Altitude Distribution', 
                    'Risk Comparison: Philadelphia vs Rio',
                    'Satellites by Orbital Zone', 
                    'Inclination vs Altitude'),
    specs=[[{'type': 'histogram'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'scatter'}]]
)

# Plot 1: Altitude distribution
fig.add_trace(
    go.Histogram(x=df_satellites['altitude_km'], nbinsx=50, 
                 name='Altitude', marker_color='blue'),
    row=1, col=1
)

# Plot 2: Risk comparison
fig.add_trace(
    go.Bar(name='Philadelphia', 
           x=['LEO Risk', 'GEO Visibility', 'Polar Coverage', 'Total Risk'], 
           y=[risk_df.iloc[0]['LEO Risk Factor'], 
              risk_df.iloc[0]['GEO Visibility Factor'],
              risk_df.iloc[0]['Polar Coverage Factor'],
              risk_df.iloc[0]['Total Risk Score']],
           marker_color='green'),
    row=1, col=2
)
fig.add_trace(
    go.Bar(name='Rio de Janeiro', 
           x=['LEO Risk', 'GEO Visibility', 'Polar Coverage', 'Total Risk'], 
           y=[risk_df.iloc[1]['LEO Risk Factor'], 
              risk_df.iloc[1]['GEO Visibility Factor'],
              risk_df.iloc[1]['Polar Coverage Factor'],
              risk_df.iloc[1]['Total Risk Score']],
           marker_color='orange'),
    row=1, col=2
)

# Plot 3: Satellites by zone
leo = len(df_satellites[df_satellites['altitude_km'] < 2000])
meo = len(df_satellites[(df_satellites['altitude_km'] >= 2000) & 
                        (df_satellites['altitude_km'] < 35786)])
geo = len(df_satellites[df_satellites['altitude_km'] >= 35786])

fig.add_trace(
    go.Bar(x=['LEO', 'MEO', 'GEO'],
           y=[leo, meo, geo],
           text=[f'{leo}', f'{meo}', f'{geo}'],
           textposition='outside',
           marker_color=['red', 'yellow', 'blue']),
    row=2, col=1
)

# Plot 4: Inclination vs Altitude scatter
fig.add_trace(
    go.Scatter(x=df_satellites['altitude_km'], 
               y=df_satellites['inclination'],
               mode='markers',
               marker=dict(size=2, opacity=0.5, color='purple'),
               name='Satellites'),
    row=2, col=2
)

# Update layout
fig.update_layout(height=800, showlegend=True, 
                  title_text="Space Situational Awareness Dashboard: Philadelphia vs Rio de Janeiro")
fig.update_xaxes(title_text="Altitude (km)", row=1, col=1)
fig.update_xaxes(title_text="Risk Category", row=1, col=2)
fig.update_xaxes(title_text="Orbit Zone", row=2, col=1)
fig.update_xaxes(title_text="Altitude (km)", row=2, col=2)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Risk Score", row=1, col=2)
fig.update_yaxes(title_text="Number of Satellites", row=2, col=1)
fig.update_yaxes(title_text="Inclination (degrees)", row=2, col=2)

fig.show()

# Summary statistics for presentation
print("\nKey Findings for Presentation:")
print(f"- Total Active Satellites Tracked: {len(df_satellites)}")
print(f"- LEO Satellites (highest collision risk): {leo}")
print(f"- Philadelphia Risk Level: {risk_df.iloc[0]['Risk Level']} (Score: {risk_df.iloc[0]['Total Risk Score']})")
print(f"- Rio de Janeiro Risk Level: {risk_df.iloc[1]['Risk Level']} (Score: {risk_df.iloc[1]['Total Risk Score']})")

Fetching satellite data...
Loaded 12525 satellites

City Risk Assessment:
             City  LEO Risk Factor  GEO Visibility Factor  Polar Coverage Factor  Total Risk Score Risk Level
0    Philadelphia            11.72                  3.298                 14.245             9.698       High
1  Rio de Janeiro            11.72                  4.421                  8.168             8.820       High





Key Findings for Presentation:
- Total Active Satellites Tracked: 12525
- LEO Satellites (highest collision risk): 11720
- Philadelphia Risk Level: High (Score: 9.698)
- Rio de Janeiro Risk Level: High (Score: 8.82)


In [18]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def create_orbital_density_map(df_satellites):
    """
    Create density heat map for collision risk prediction
    """
    # Define orbital zones (altitude bands in km)
    altitude_bins = np.arange(200, 2000, 50)  # LEO in 50km increments
    inclination_bins = np.arange(0, 180, 5)   # 5-degree increments
    
    # Create 2D histogram for density
    density_matrix = np.histogram2d(
        df_satellites['altitude_km'],
        df_satellites['inclination'],
        bins=[altitude_bins, inclination_bins]
    )[0]
    
    # Apply debris multiplication factor based on historical data
    # Real debris is 10x satellites in LEO
    debris_factor = np.where(altitude_bins[:-1] < 1000, 10, 1)
    debris_density = density_matrix * debris_factor.reshape(-1, 1)
    
    return debris_density, altitude_bins, inclination_bins

def calculate_collision_probability(density_matrix, altitude_bins):
    """
    Calculate collision probability based on density and orbital velocity
    """
    # Orbital velocity decreases with altitude
    # v = sqrt(GM/r) where r = Earth_radius + altitude
    velocities = np.sqrt(398600.5 / (6371 + altitude_bins[:-1]))
    
    # Collision probability increases with density squared (pairwise interactions)
    # and with relative velocity
    collision_risk = density_matrix**2 * velocities.reshape(-1, 1) / 1000
    
    return collision_risk

def predict_future_density(current_density, years=5):
    """
    Predict future debris growth using Kessler Syndrome model
    """
    # Simplified exponential growth in high-density regions
    growth_rate = np.where(current_density > 50, 1.15, 1.05)  # 15% vs 5% annual growth
    future_density = current_density * (growth_rate ** years)
    
    return future_density

In [19]:
# Create current and predicted density maps
current_density, alt_bins, inc_bins = create_orbital_density_map(df_satellites)
collision_risk = calculate_collision_probability(current_density, alt_bins)
future_density = predict_future_density(current_density, years=5)
future_risk = calculate_collision_probability(future_density, alt_bins)

# Calculate risk scores for cities based on satellites passing through high-risk zones
def calculate_city_debris_risk(city_lat, city_lon, density_matrix, inc_bins):
    """
    Calculate debris risk for a specific city based on orbital passages
    """
    # Satellites visible from city predominantly have inclinations >= abs(latitude)
    min_inclination = abs(city_lat)
    
    # Find which inclination bands affect this city
    relevant_inc_indices = np.where(inc_bins[:-1] >= min_inclination)[0]
    
    # Sum risk from relevant orbital bands
    if len(relevant_inc_indices) > 0:
        city_risk = np.sum(density_matrix[:, relevant_inc_indices])
    else:
        city_risk = 0
    
    # Normalize to 0-100 scale
    return min(100, city_risk / 10)

# Calculate for both cities
philly_current_risk = calculate_city_debris_risk(39.95, -75.16, collision_risk, inc_bins)
rio_current_risk = calculate_city_debris_risk(-22.91, -43.17, collision_risk, inc_bins)
philly_future_risk = calculate_city_debris_risk(39.95, -75.16, future_risk, inc_bins)
rio_future_risk = calculate_city_debris_risk(-22.91, -43.17, future_risk, inc_bins)

In [21]:
# Create comprehensive dashboard with predictions - WHITE BACKGROUND VERSION
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Current Debris Density Heat Map', 
                    '5-Year Predicted Density (Kessler Effect)',
                    'City Risk Score Projection', 
                    'High-Risk Orbital Zones'),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}],
           [{'type': 'scatter'}, {'type': 'bar'}]]
)

# Current density heatmap with better color scale
fig.add_trace(
    go.Heatmap(z=current_density.T, 
               x=alt_bins[:-1], 
               y=inc_bins[:-1],
               colorscale='Viridis',  # Changed from 'Hot' for better visibility
               colorbar=dict(title="Density"),
               text=np.round(current_density.T, 1),
               hovertemplate='Altitude: %{x}km<br>Inclination: %{y}°<br>Density: %{z}<extra></extra>'),
    row=1, col=1
)

# Predicted future density with same color scale
fig.add_trace(
    go.Heatmap(z=future_density.T, 
               x=alt_bins[:-1], 
               y=inc_bins[:-1],
               colorscale='Viridis',
               colorbar=dict(title="Density"),
               text=np.round(future_density.T, 1),
               hovertemplate='Altitude: %{x}km<br>Inclination: %{y}°<br>Density: %{z}<extra></extra>'),
    row=1, col=2
)

# Risk projection over time
years = np.arange(0, 11)
philly_projection = philly_current_risk * (1.12 ** years)
rio_projection = rio_current_risk * (1.10 ** years)

fig.add_trace(
    go.Scatter(x=years, y=philly_projection, 
               mode='lines+markers', name='Philadelphia',
               line=dict(width=3, color='blue'),
               marker=dict(size=8)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=years, y=rio_projection, 
               mode='lines+markers', name='Rio de Janeiro',
               line=dict(width=3, color='green'),
               marker=dict(size=8)),
    row=2, col=1
)

# Highest risk zones
high_risk_zones = []
for i in range(len(alt_bins)-1):
    max_risk_inc = np.argmax(collision_risk[i])
    high_risk_zones.append({
        'altitude': alt_bins[i],
        'risk_score': np.max(collision_risk[i])
    })
top_zones = sorted(high_risk_zones, key=lambda x: x['risk_score'], reverse=True)[:5]

fig.add_trace(
    go.Bar(x=[f"{z['altitude']:.0f}km" for z in top_zones],
           y=[z['risk_score'] for z in top_zones],
           marker_color='darkred',
           text=[f"{z['risk_score']:.1f}" for z in top_zones],
           textposition='outside'),
    row=2, col=2
)

# Update layout with white background and better visibility
fig.update_layout(
    height=800, 
    showlegend=True,
    title_text="Predictive Debris Risk Analysis: Kessler Syndrome Model",
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black', size=12)
)

# Update axes for better visibility
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', 
                 zeroline=True, zerolinewidth=2, zerolinecolor='lightgray',
                 title_font=dict(size=14))
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray',
                 zeroline=True, zerolinewidth=2, zerolinecolor='lightgray',
                 title_font=dict(size=14))

# Add specific axis labels
fig.update_xaxes(title_text="Altitude (km)", row=1, col=1)
fig.update_xaxes(title_text="Altitude (km)", row=1, col=2)
fig.update_xaxes(title_text="Years from Present", row=2, col=1)
fig.update_xaxes(title_text="Altitude Zone", row=2, col=2)

fig.update_yaxes(title_text="Inclination (degrees)", row=1, col=1)
fig.update_yaxes(title_text="Inclination (degrees)", row=1, col=2)
fig.update_yaxes(title_text="Risk Score (0-100)", row=2, col=1)
fig.update_yaxes(title_text="Collision Risk Index", row=2, col=2)

fig.show()

In [ ]:
def parse_tle_with_risk_metrics(tle_text):
    """
    Parse TLE data and calculate risk metrics
    TLE CANNOT provide debris risk or collision scores directly - we calculate them
    """
    satellites = []
    lines = tle_text.strip().split('\n')
    
    for i in range(0, len(lines)-2, 3):
        try:
            name = lines[i].strip()
            line1 = lines[i+1].strip()
            line2 = lines[i+2].strip()
            
            # Extract TLE parameters
            satellite_data = {
                'name': name,
                'norad_id': int(line1[2:7]),
                'classification': line1[7],  # U=unclassified, C=classified
                'launch_year': int(line1[9:11]),
                'launch_number': int(line1[11:14]),
                'piece_designation': line1[14:17].strip(),
                'epoch_year': int(line1[18:20]),
                'epoch_day': float(line1[20:32]),
                'mean_motion_derivative': float(line1[33:43]),
                'inclination': float(line2[8:16]),
                'raan': float(line2[17:25]),  # Right Ascension
                'eccentricity': float('0.' + line2[26:33]),
                'arg_perigee': float(line2[34:42]),
                'mean_anomaly': float(line2[43:51]),
                'mean_motion': float(line2[52:63]),
                'revolution_number': int(line2[63:68])
            }
            
            # Calculate derived metrics
            satellite_data.update(calculate_orbital_metrics(satellite_data))
            
            # Calculate risk scores (these are derived, not in TLE)
            satellite_data.update(calculate_risk_scores(satellite_data))
            
            satellites.append(satellite_data)
        except Exception as e:
            continue
    
    return pd.DataFrame(satellites)

def calculate_orbital_metrics(sat_data):
    """
    Calculate altitude and other metrics from TLE orbital elements
    """
    # Constants
    EARTH_RADIUS_KM = 6371
    MU = 398600.4418  # Earth's gravitational parameter
    
    # Calculate orbital period in minutes
    mean_motion = sat_data['mean_motion']
    period_minutes = 1440.0 / mean_motion
    
    # Calculate semi-major axis and altitude
    # Using Kepler's third law: n = sqrt(μ/a³)
    a = (MU / (mean_motion * 2 * np.pi / 86400) ** 2) ** (1/3)
    
    # Calculate perigee and apogee
    eccentricity = sat_data['eccentricity']
    perigee_km = a * (1 - eccentricity) - EARTH_RADIUS_KM
    apogee_km = a * (1 + eccentricity) - EARTH_RADIUS_KM
    mean_altitude_km = a - EARTH_RADIUS_KM
    
    return {
        'period_minutes': period_minutes,
        'semi_major_axis_km': a,
        'perigee_km': perigee_km,
        'apogee_km': apogee_km,
        'mean_altitude_km': mean_altitude_km,
        'orbit_class': classify_orbit(mean_altitude_km)
    }

def classify_orbit(altitude_km):
    """Classify orbit type based on altitude"""
    if altitude_km < 2000:
        return 'LEO'
    elif altitude_km < 35786:
        return 'MEO'
    elif 35786 <= altitude_km < 36286:
        return 'GEO'
    else:
        return 'HEO'

def calculate_risk_scores(sat_data):
    """
    Calculate debris and collision risk scores
    IMPORTANT: These are DERIVED metrics, not in TLE data
    """
    
    # 1. DEBRIS RISK SCORE
    # Based on altitude (known debris fields) and age
    altitude = sat_data['mean_altitude_km']
    
    # Known debris zones (from historical events)
    debris_risk = 0.0
    
    # Chinese ASAT test debris zone
    if 850 <= altitude <= 900:
        debris_risk += 0.3
    
    # Iridium-Cosmos collision zone
    if 770 <= altitude <= 820:
        debris_risk += 0.25
    
    # General LEO congestion
    if 400 <= altitude <= 600:
        debris_risk += 0.2
    elif 600 <= altitude <= 1000:
        debris_risk += 0.15
    
    # Piece designation indicates debris
    if sat_data['piece_designation'] not in ['A', 'B']:  # A/B are usually payloads
        debris_risk += 0.2
    
    # 2. COLLISION PROBABILITY SCORE
    # Based on orbital parameters
    collision_score = 0.0
    
    # High inclination variation = crossing orbits
    inclination = sat_data['inclination']
    if 50 <= inclination <= 60:  # High traffic inclinations
        collision_score += 0.2
    if 96 <= inclination <= 100:  # Sun-synchronous (very crowded)
        collision_score += 0.3
    
    # Eccentricity (elliptical orbits cross multiple altitude bands)
    if sat_data['eccentricity'] > 0.01:
        collision_score += sat_data['eccentricity'] * 10
    
    # High mean motion derivative indicates orbital decay
    if abs(sat_data['mean_motion_derivative']) > 0.0001:
        collision_score += 0.1
    
    return {
        'debris_risk_score': min(1.0, debris_risk),
        'collision_probability_score': min(1.0, collision_score),
        'combined_risk_score': min(1.0, (debris_risk + collision_score) / 2)
    }